<a href="https://colab.research.google.com/github/AndrePalheta3008/Deep-Learning/blob/main/Andre_202200480001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AMOSTRA 1
#X [0.0,1]
#Y [-1,-1.3]


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from keras import backend as k
from keras import losses
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error

#Initialize all weights to match the values adopted in the example at
#https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/ 
#Initialise kernel (weights matrix) to required value
def kernel_init_layer1(shape,dtype='float'):
    # In this simple example, assume that the shape is (2, 2) 
    # and create a fixed array with this dimension for layer 1
    kernel = np.array([[0.1, 0.3],[-0.4, 0.2]])
    return kernel 

#Initialise kernel (weights matrix) to required value
def kernel_init_layer2(shape,dtype='float'):
    # In this simple example, assume that the shape is (2, 2)
    # and create a fixed array with this dimension for layer 2
    kernel = np.array([[0.4, 0.2], [-0.4, 0.7]])
    return kernel 

#Initialise kernel (bias vector) to required value
def kernel_init_bias1(shape,dtype='float'):
    # In this simple example, assume that the shape is (1, 2)
    # and create a fixed array with this dimension for the 
    # bias vector of layer 1
    kernel = np.array([-0.4, 0.8]) #both neurons with the same value
    return kernel 

#Initialise kernel (bias vector) to required value
def kernel_init_bias2(shape,dtype='float'):
    # In this simple example, assume that the shape is (1, 2)
    # and create a fixed array with this dimension for the 
    # bias vector of layer 2
    kernel = np.array([0.8, 0.2]) #both neurons with the same value
    return kernel 

# from https://stackoverflow.com/questions/66221788/tf-gradients-is-not-supported-when-eager-execution-is-enabled-use-tf-gradientta
# TF 2 does not use "eager" execution, so disable it:
tf.compat.v1.disable_eager_execution()

#Define the neural network model with dense layers. Syntax:
#https://keras.io/api/layers/core_layers/dense/
#The sigmoid activation function in Keras is the standard logistic function 1/(1+exp(-x)).
#https://keras.io/api/layers/activations/
model = Sequential()
model.add(Dense(2, input_dim=2, use_bias=True,  bias_initializer=kernel_init_bias1,
        kernel_initializer=kernel_init_layer1, activation='sigmoid'))
model.add(Dense(2, use_bias=True,  bias_initializer=kernel_init_bias2, 
        kernel_initializer=kernel_init_layer2, activation='sigmoid'))
model.summary() # display the architecture

# We are not going to use a Keras optimizer. Define a learning rate:
learning_rate = 1  #you can change to 0.5 or any other reasonable value

#now compile the model, informing loss and performanc metrics that
#should be computed along the training
model.compile(loss='mse', metrics=['accuracy'])

# Begin TensorFlow
sess = tf.compat.v1.InteractiveSession()
sess.run(tf.compat.v1.initialize_all_variables())

model.optimizer.lr = learning_rate

print("===START TRAINING THE NETWORK:===")
steps = 3 # steps of gradient descent
for s in range(steps):
    print('Learning rate = ', k.get_value(model.optimizer.lr))
    print(" ############ Step = " + str(s) + " ############")
    print("1) ===BEFORE using any GRADIENT:===")
    #define input and target vectors, and also inform it to the loss function object
    #in this code we will keep the same inputs and targets along the steps, but the inputs and
    #targets can change when we go, for instance, through the training set
    inputs = np.array([[0.0,1]]) #define the input for the current iteration (step)
    outputs = model.predict(inputs) #forward pass
    #define the target vector for the current iteration (step)
    targets = np.array([[-1, -1.3]]) #notice that a sigmoid can output within range [0, 1]
    mse = mean_squared_error(targets, outputs) #calculate MSE
    #initialize loss object to be later incorporated to the gradients object that
    #enables the calculation of the symbolic gradients
    loss = losses.mean_squared_error(targets, model.output)
    #  ===== Obtain symbolic gradient to calculate numerical gradients =====
    gradients = k.gradients(loss, model.trainable_weights) #inform loss and weights
    if False: #enable with True in case you want to see the objects
        print("List of tensors representing the symbolic gradients:")
        for i in range(len(gradients)):
            print('symbolic gradient[',i,']=',gradients[i])
    print('Network input [x1, x2]:\n', inputs)
    print('Network output [out_o1, out_o2]:\n', outputs)
    print("targets:\n", targets)
    #show weights at the beginning of iteration s
    print('weights at the beginning of this step')
    for i in range(len(model.trainable_weights)):
        #note that model.trainable_weights[i] is an object of
        # <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable
        #therefore (see e.g. https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable )
        #you need to obtain its value via a TF session:
        print('weights[',i,']=',sess.run(model.trainable_weights[i]))
    print("MSE with the initial weights:", mse)

    print("2) ===After applying GRADIENT:===")
    print("------------- Results for step (iteration) =", s)
    # ===== Calculate numerical gradient from symbolic gradients =====
    # evaluated_gradients is a list, show its contents:
    evaluated_gradients = sess.run(gradients, feed_dict={model.input: inputs})
    print('Gradients g to be used in new_weights = current_weights - learning_rate*g')
    for i in range(len(evaluated_gradients)):
        print('gradients[',i,']=',evaluated_gradients[i])

    # Apply ("step down") the gradient for each layer, subtracting the gradients
    # from current weights scaled by the learning rate:
    for i in range(len(model.trainable_weights)):
        sess.run(tf.compat.v1.assign_sub(model.trainable_weights[i], learning_rate*evaluated_gradients[i]))

    #show weights after gradient propagation of iteration s
    print('weights after gradient propagation in this step')
    for i in range(len(model.trainable_weights)):
        print('weights[',i,']=',sess.run(model.trainable_weights[i]))

    # print the MSE with new weights:
    outputs = model.predict(inputs)
    mse = mean_squared_error(targets, outputs)
    print("MSE with the new weights:", mse)

#Collect and show final results
final_outputs = model.predict(inputs)
final_mse = mean_squared_error(targets, final_outputs)

print("\n ===AFTER executing all GRADIENT descent steps===")
#show weights at the end of training
for i in range(len(model.trainable_weights)):
    #note that model.trainable_weights[i] is an object of
    # <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable
    #therefore (see e.g. https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable )
    #you need to obtain its value via a TF session:
    print('final weights[',i,']=',sess.run(model.trainable_weights[i]))
print("outputs:\n", final_outputs)
print("targets:\n", targets)
print("Final MSE = ", final_mse)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 2)                 6         
                                                                 
 dense_3 (Dense)             (None, 2)                 6         
                                                                 
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________
===START TRAINING THE NETWORK:===
Learning rate =  1
 ############ Step = 0 ############
1) ===BEFORE using any GRADIENT:===
Network input [x1, x2]:
 [[0. 1.]]
Network output [out_o1, out_o2]:
 [[0.65284914 0.6843307 ]]
targets:
 [[-1.  -1.3]]
weights at the beginning of this step
weights[ 0 ]= [[ 0.1  0.3]
 [-0.4  0.2]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


weights[ 1 ]= [-0.4  0.8]
weights[ 2 ]= [[ 0.4  0.2]
 [-0.4  0.7]]
weights[ 3 ]= [0.8 0.2]
MSE with the initial weights: 3.334739303243694
2) ===After applying GRADIENT:===
------------- Results for step (iteration) = 0
Gradients g to be used in new_weights = current_weights - learning_rate*g
gradients[ 0 ]= [[0.         0.        ]
 [0.05039085 0.0295356 ]]
gradients[ 1 ]= [0.05039085 0.0295356 ]
gradients[ 2 ]= [[0.11613464 0.13289538]
 [0.27385238 0.3133752 ]]
gradients[ 3 ]= [0.374597   0.42865947]
weights after gradient propagation in this step
weights[ 0 ]= [[ 0.1         0.3       ]
 [-0.45039088  0.17046441]]
weights[ 1 ]= [-0.45039088  0.7704644 ]
weights[ 2 ]= [[ 0.28386536  0.06710462]
 [-0.6738524   0.38662478]]
weights[ 3 ]= [ 0.425403   -0.22865947]
MSE with the new weights: 2.7846389974415917
Learning rate =  1
 ############ Step = 1 ############
1) ===BEFORE using any GRADIENT:===
Network input [x1, x2]:
 [[0. 1.]]
Network output [out_o1, out_o2]:
 [[0.5056786 0.5171984

#AMOSTRA 2
#X 0.0,-1
#x 1,0.9

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from keras import backend as k
from keras import losses
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error

#Initialize all weights to match the values adopted in the example at
#https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/ 
#Initialise kernel (weights matrix) to required value
def kernel_init_layer1(shape,dtype='float'):
    # In this simple example, assume that the shape is (2, 2) 
    # and create a fixed array with this dimension for layer 1
    kernel = np.array([[0.1, 0.3],[-0.4, 0.2]])
    return kernel 

#Initialise kernel (weights matrix) to required value
def kernel_init_layer2(shape,dtype='float'):
    # In this simple example, assume that the shape is (2, 2)
    # and create a fixed array with this dimension for layer 2
    kernel = np.array([[0.4, 0.2], [-0.4, 0.7]])
    return kernel 

#Initialise kernel (bias vector) to required value
def kernel_init_bias1(shape,dtype='float'):
    # In this simple example, assume that the shape is (1, 2)
    # and create a fixed array with this dimension for the 
    # bias vector of layer 1
    kernel = np.array([-0.4, 0.8]) #both neurons with the same value
    return kernel 

#Initialise kernel (bias vector) to required value
def kernel_init_bias2(shape,dtype='float'):
    # In this simple example, assume that the shape is (1, 2)
    # and create a fixed array with this dimension for the 
    # bias vector of layer 2
    kernel = np.array([0.8, 0.2]) #both neurons with the same value
    return kernel 

# from https://stackoverflow.com/questions/66221788/tf-gradients-is-not-supported-when-eager-execution-is-enabled-use-tf-gradientta
# TF 2 does not use "eager" execution, so disable it:
tf.compat.v1.disable_eager_execution()

#Define the neural network model with dense layers. Syntax:
#https://keras.io/api/layers/core_layers/dense/
#The sigmoid activation function in Keras is the standard logistic function 1/(1+exp(-x)).
#https://keras.io/api/layers/activations/
model = Sequential()
model.add(Dense(2, input_dim=2, use_bias=True,  bias_initializer=kernel_init_bias1,
        kernel_initializer=kernel_init_layer1, activation='sigmoid'))
model.add(Dense(2, use_bias=True,  bias_initializer=kernel_init_bias2, 
        kernel_initializer=kernel_init_layer2, activation='sigmoid'))
model.summary() # display the architecture

# We are not going to use a Keras optimizer. Define a learning rate:
learning_rate = 1  #you can change to 0.5 or any other reasonable value

#now compile the model, informing loss and performanc metrics that
#should be computed along the training
model.compile(loss='mse', metrics=['accuracy'])

# Begin TensorFlow
sess = tf.compat.v1.InteractiveSession()
sess.run(tf.compat.v1.initialize_all_variables())

model.optimizer.lr = learning_rate

print("===START TRAINING THE NETWORK:===")
steps = 3 # steps of gradient descent
for s in range(steps):
    print('Learning rate = ', k.get_value(model.optimizer.lr))
    print(" ############ Step = " + str(s) + " ############")
    print("1) ===BEFORE using any GRADIENT:===")
    #define input and target vectors, and also inform it to the loss function object
    #in this code we will keep the same inputs and targets along the steps, but the inputs and
    #targets can change when we go, for instance, through the training set
    inputs = np.array([[0.0,-1]]) #define the input for the current iteration (step)
    outputs = model.predict(inputs) #forward pass
    #define the target vector for the current iteration (step)
    targets = np.array([[1, 0.9]]) #notice that a sigmoid can output within range [0, 1]
    mse = mean_squared_error(targets, outputs) #calculate MSE
    #initialize loss object to be later incorporated to the gradients object that
    #enables the calculation of the symbolic gradients
    loss = losses.mean_squared_error(targets, model.output)
    #  ===== Obtain symbolic gradient to calculate numerical gradients =====
    gradients = k.gradients(loss, model.trainable_weights) #inform loss and weights
    if False: #enable with True in case you want to see the objects
        print("List of tensors representing the symbolic gradients:")
        for i in range(len(gradients)):
            print('symbolic gradient[',i,']=',gradients[i])
    print('Network input [x1, x2]:\n', inputs)
    print('Network output [out_o1, out_o2]:\n', outputs)
    print("targets:\n", targets)
    #show weights at the beginning of iteration s
    print('weights at the beginning of this step')
    for i in range(len(model.trainable_weights)):
        #note that model.trainable_weights[i] is an object of
        # <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable
        #therefore (see e.g. https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable )
        #you need to obtain its value via a TF session:
        print('weights[',i,']=',sess.run(model.trainable_weights[i]))
    print("MSE with the initial weights:", mse)

    print("2) ===After applying GRADIENT:===")
    print("------------- Results for step (iteration) =", s)
    # ===== Calculate numerical gradient from symbolic gradients =====
    # evaluated_gradients is a list, show its contents:
    evaluated_gradients = sess.run(gradients, feed_dict={model.input: inputs})
    print('Gradients g to be used in new_weights = current_weights - learning_rate*g')
    for i in range(len(evaluated_gradients)):
        print('gradients[',i,']=',evaluated_gradients[i])

    # Apply ("step down") the gradient for each layer, subtracting the gradients
    # from current weights scaled by the learning rate:
    for i in range(len(model.trainable_weights)):
        sess.run(tf.compat.v1.assign_sub(model.trainable_weights[i], learning_rate*evaluated_gradients[i]))

    #show weights after gradient propagation of iteration s
    print('weights after gradient propagation in this step')
    for i in range(len(model.trainable_weights)):
        print('weights[',i,']=',sess.run(model.trainable_weights[i]))

    # print the MSE with new weights:
    outputs = model.predict(inputs)
    mse = mean_squared_error(targets, outputs)
    print("MSE with the new weights:", mse)

#Collect and show final results
final_outputs = model.predict(inputs)
final_mse = mean_squared_error(targets, final_outputs)

print("\n ===AFTER executing all GRADIENT descent steps===")
#show weights at the end of training
for i in range(len(model.trainable_weights)):
    #note that model.trainable_weights[i] is an object of
    # <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable
    #therefore (see e.g. https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable )
    #you need to obtain its value via a TF session:
    print('final weights[',i,']=',sess.run(model.trainable_weights[i]))
print("outputs:\n", final_outputs)
print("targets:\n", targets)
print("Final MSE = ", final_mse)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 2)                 6         
                                                                 
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


Instructions for updating:
Use `tf.global_variables_initializer` instead.
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


===START TRAINING THE NETWORK:===
Learning rate =  1
 ############ Step = 0 ############
1) ===BEFORE using any GRADIENT:===
Network input [x1, x2]:
 [[ 0. -1.]]
Network output [out_o1, out_o2]:
 [[0.6773757 0.6796055]]
targets:
 [[1.  0.9]]
weights at the beginning of this step
weights[ 0 ]= [[ 0.1  0.3]
 [-0.4  0.2]]
weights[ 1 ]= [-0.4  0.8]
weights[ 2 ]= [[ 0.4  0.2]
 [-0.4  0.7]]
weights[ 3 ]= [0.8 0.2]
MSE with the initial weights: 0.07633009912304971
2) ===After applying GRADIENT:===
------------- Results for step (iteration) = 0
Gradients g to be used in new_weights = current_weights - learning_rate*g
gradients[ 0 ]= [[-0.         -0.        ]
 [ 0.00945002  0.00123317]]
gradients[ 1 ]= [-0.00945002 -0.00123317]
gradients[ 2 ]= [[-0.03525282 -0.02399455]
 [-0.04552241 -0.03098447]]
gradients[ 3 ]= [-0.07050563 -0.04798911]
weights after gradient propagation in this step
weights[ 0 ]= [[ 0.1         0.3       ]
 [-0.40945002  0.19876683]]
weights[ 1 ]= [-0.39055    0.8012332]
we

#Amostra 3

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from keras import backend as k
from keras import losses
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error

#Initialize all weights to match the values adopted in the example at
#https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/ 
#Initialise kernel (weights matrix) to required value
def kernel_init_layer1(shape,dtype='float'):
    # In this simple example, assume that the shape is (2, 2) 
    # and create a fixed array with this dimension for layer 1
    kernel = np.array([[0.1, 0.3],[-0.4, 0.2]])
    return kernel 

#Initialise kernel (weights matrix) to required value
def kernel_init_layer2(shape,dtype='float'):
    # In this simple example, assume that the shape is (2, 2)
    # and create a fixed array with this dimension for layer 2
    kernel = np.array([[0.4, 0.2], [-0.4, 0.7]])
    return kernel 

#Initialise kernel (bias vector) to required value
def kernel_init_bias1(shape,dtype='float'):
    # In this simple example, assume that the shape is (1, 2)
    # and create a fixed array with this dimension for the 
    # bias vector of layer 1
    kernel = np.array([-0.4, 0.8]) #both neurons with the same value
    return kernel 

#Initialise kernel (bias vector) to required value
def kernel_init_bias2(shape,dtype='float'):
    # In this simple example, assume that the shape is (1, 2)
    # and create a fixed array with this dimension for the 
    # bias vector of layer 2
    kernel = np.array([0.8, 0.2]) #both neurons with the same value
    return kernel 

# from https://stackoverflow.com/questions/66221788/tf-gradients-is-not-supported-when-eager-execution-is-enabled-use-tf-gradientta
# TF 2 does not use "eager" execution, so disable it:
tf.compat.v1.disable_eager_execution()

#Define the neural network model with dense layers. Syntax:
#https://keras.io/api/layers/core_layers/dense/
#The sigmoid activation function in Keras is the standard logistic function 1/(1+exp(-x)).
#https://keras.io/api/layers/activations/
model = Sequential()
model.add(Dense(2, input_dim=2, use_bias=True,  bias_initializer=kernel_init_bias1,
        kernel_initializer=kernel_init_layer1, activation='sigmoid'))
model.add(Dense(2, use_bias=True,  bias_initializer=kernel_init_bias2, 
        kernel_initializer=kernel_init_layer2, activation='sigmoid'))
model.summary() # display the architecture

# We are not going to use a Keras optimizer. Define a learning rate:
learning_rate = 1  #you can change to 0.5 or any other reasonable value

#now compile the model, informing loss and performanc metrics that
#should be computed along the training
model.compile(loss='mse', metrics=['accuracy'])

# Begin TensorFlow
sess = tf.compat.v1.InteractiveSession()
sess.run(tf.compat.v1.initialize_all_variables())

model.optimizer.lr = learning_rate

print("===START TRAINING THE NETWORK:===")
steps = 3 # steps of gradient descent
for s in range(steps):
    print('Learning rate = ', k.get_value(model.optimizer.lr))
    print(" ############ Step = " + str(s) + " ############")
    print("1) ===BEFORE using any GRADIENT:===")
    #define input and target vectors, and also inform it to the loss function object
    #in this code we will keep the same inputs and targets along the steps, but the inputs and
    #targets can change when we go, for instance, through the training set
    inputs = np.array([[1.2,0]]) #define the input for the current iteration (step)
    outputs = model.predict(inputs) #forward pass
    #define the target vector for the current iteration (step)
    targets = np.array([[1, 0.9]]) #notice that a sigmoid can output within range [0, 1]
    mse = mean_squared_error(targets, outputs) #calculate MSE
    #initialize loss object to be later incorporated to the gradients object that
    #enables the calculation of the symbolic gradients
    loss = losses.mean_squared_error(targets, model.output)
    #  ===== Obtain symbolic gradient to calculate numerical gradients =====
    gradients = k.gradients(loss, model.trainable_weights) #inform loss and weights
    if False: #enable with True in case you want to see the objects
        print("List of tensors representing the symbolic gradients:")
        for i in range(len(gradients)):
            print('symbolic gradient[',i,']=',gradients[i])
    print('Network input [x1, x2]:\n', inputs)
    print('Network output [out_o1, out_o2]:\n', outputs)
    print("targets:\n", targets)
    #show weights at the beginning of iteration s
    print('weights at the beginning of this step')
    for i in range(len(model.trainable_weights)):
        #note that model.trainable_weights[i] is an object of
        # <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable
        #therefore (see e.g. https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable )
        #you need to obtain its value via a TF session:
        print('weights[',i,']=',sess.run(model.trainable_weights[i]))
    print("MSE with the initial weights:", mse)

    print("2) ===After applying GRADIENT:===")
    print("------------- Results for step (iteration) =", s)
    # ===== Calculate numerical gradient from symbolic gradients =====
    # evaluated_gradients is a list, show its contents:
    evaluated_gradients = sess.run(gradients, feed_dict={model.input: inputs})
    print('Gradients g to be used in new_weights = current_weights - learning_rate*g')
    for i in range(len(evaluated_gradients)):
        print('gradients[',i,']=',evaluated_gradients[i])

    # Apply ("step down") the gradient for each layer, subtracting the gradients
    # from current weights scaled by the learning rate:
    for i in range(len(model.trainable_weights)):
        sess.run(tf.compat.v1.assign_sub(model.trainable_weights[i], learning_rate*evaluated_gradients[i]))

    #show weights after gradient propagation of iteration s
    print('weights after gradient propagation in this step')
    for i in range(len(model.trainable_weights)):
        print('weights[',i,']=',sess.run(model.trainable_weights[i]))

    # print the MSE with new weights:
    outputs = model.predict(inputs)
    mse = mean_squared_error(targets, outputs)
    print("MSE with the new weights:", mse)

#Collect and show final results
final_outputs = model.predict(inputs)
final_mse = mean_squared_error(targets, final_outputs)

print("\n ===AFTER executing all GRADIENT descent steps===")
#show weights at the end of training
for i in range(len(model.trainable_weights)):
    #note that model.trainable_weights[i] is an object of
    # <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable
    #therefore (see e.g. https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable )
    #you need to obtain its value via a TF session:
    print('final weights[',i,']=',sess.run(model.trainable_weights[i]))
print("outputs:\n", final_outputs)
print("targets:\n", targets)
print("Final MSE = ", final_mse)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 2)                 6         
                                                                 
 dense_3 (Dense)             (None, 2)                 6         
                                                                 
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________
===START TRAINING THE NETWORK:===
Learning rate =  1
 ############ Step = 0 ############
1) ===BEFORE using any GRADIENT:===
Network input [x1, x2]:
 [[1.2 0. ]]
Network output [out_o1, out_o2]:
 [[0.6609764 0.6940291]]
targets:
 [[1.  0.9]]
weights at the beginning of this step
weights[ 0 ]= [[ 0.1  0.3]
 [-0.4  0.2]]
weights[ 1 ]= [-0.4  0.8]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


weights[ 2 ]= [[ 0.4  0.2]
 [-0.4  0.7]]
weights[ 3 ]= [0.8 0.2]
MSE with the initial weights: 0.07868050462676479
2) ===After applying GRADIENT:===
------------- Results for step (iteration) = 0
Gradients g to be used in new_weights = current_weights - learning_rate*g
gradients[ 0 ]= [[-1.1513641e-02 -4.9862119e-05]
 [-0.0000000e+00 -0.0000000e+00]]
gradients[ 1 ]= [-9.5947003e-03 -4.1551764e-05]
gradients[ 2 ]= [[-0.03270185 -0.01882739]
 [-0.05783894 -0.03329954]]
gradients[ 3 ]= [-0.07597064 -0.04373848]
weights after gradient propagation in this step
weights[ 0 ]= [[ 0.11151364  0.30004987]
 [-0.4         0.2       ]]
weights[ 1 ]= [-0.3904053   0.80004156]
weights[ 2 ]= [[ 0.43270186  0.2188274 ]
 [-0.34216106  0.7332995 ]]
weights[ 3 ]= [0.87597066 0.24373847]
MSE with the new weights: 0.06574698873984872
Learning rate =  1
 ############ Step = 1 ############
1) ===BEFORE using any GRADIENT:===
Network input [x1, x2]:
 [[1.2 0. ]]
Network output [out_o1, out_o2]:
 [[0.69087464 0

# AMOSTRA 1 com rate 0.5

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from keras import backend as k
from keras import losses
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error

#Initialize all weights to match the values adopted in the example at
#https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/ 
#Initialise kernel (weights matrix) to required value
def kernel_init_layer1(shape,dtype='float'):
    # In this simple example, assume that the shape is (2, 2) 
    # and create a fixed array with this dimension for layer 1
    kernel = np.array([[0.1, 0.3],[-0.4, 0.2]])
    return kernel 

#Initialise kernel (weights matrix) to required value
def kernel_init_layer2(shape,dtype='float'):
    # In this simple example, assume that the shape is (2, 2)
    # and create a fixed array with this dimension for layer 2
    kernel = np.array([[0.4, 0.2], [-0.4, 0.7]])
    return kernel 

#Initialise kernel (bias vector) to required value
def kernel_init_bias1(shape,dtype='float'):
    # In this simple example, assume that the shape is (1, 2)
    # and create a fixed array with this dimension for the 
    # bias vector of layer 1
    kernel = np.array([-0.4, 0.8]) #both neurons with the same value
    return kernel 

#Initialise kernel (bias vector) to required value
def kernel_init_bias2(shape,dtype='float'):
    # In this simple example, assume that the shape is (1, 2)
    # and create a fixed array with this dimension for the 
    # bias vector of layer 2
    kernel = np.array([0.8, 0.2]) #both neurons with the same value
    return kernel 

# from https://stackoverflow.com/questions/66221788/tf-gradients-is-not-supported-when-eager-execution-is-enabled-use-tf-gradientta
# TF 2 does not use "eager" execution, so disable it:
tf.compat.v1.disable_eager_execution()

#Define the neural network model with dense layers. Syntax:
#https://keras.io/api/layers/core_layers/dense/
#The sigmoid activation function in Keras is the standard logistic function 1/(1+exp(-x)).
#https://keras.io/api/layers/activations/
model = Sequential()
model.add(Dense(2, input_dim=2, use_bias=True,  bias_initializer=kernel_init_bias1,
        kernel_initializer=kernel_init_layer1, activation='sigmoid'))
model.add(Dense(2, use_bias=True,  bias_initializer=kernel_init_bias2, 
        kernel_initializer=kernel_init_layer2, activation='sigmoid'))
model.summary() # display the architecture

# We are not going to use a Keras optimizer. Define a learning rate:
learning_rate = 0.5  #you can change to 0.5 or any other reasonable value

#now compile the model, informing loss and performanc metrics that
#should be computed along the training
model.compile(loss='mse', metrics=['accuracy'])

# Begin TensorFlow
sess = tf.compat.v1.InteractiveSession()
sess.run(tf.compat.v1.initialize_all_variables())

model.optimizer.lr = learning_rate

print("===START TRAINING THE NETWORK:===")
steps = 3 # steps of gradient descent
for s in range(steps):
    print('Learning rate = ', k.get_value(model.optimizer.lr))
    print(" ############ Step = " + str(s) + " ############")
    print("1) ===BEFORE using any GRADIENT:===")
    #define input and target vectors, and also inform it to the loss function object
    #in this code we will keep the same inputs and targets along the steps, but the inputs and
    #targets can change when we go, for instance, through the training set
    inputs = np.array([[0.0,1]]) #define the input for the current iteration (step)
    outputs = model.predict(inputs) #forward pass
    #define the target vector for the current iteration (step)
    targets = np.array([[-1, -1.3]]) #notice that a sigmoid can output within range [0, 1]
    mse = mean_squared_error(targets, outputs) #calculate MSE
    #initialize loss object to be later incorporated to the gradients object that
    #enables the calculation of the symbolic gradients
    loss = losses.mean_squared_error(targets, model.output)
    #  ===== Obtain symbolic gradient to calculate numerical gradients =====
    gradients = k.gradients(loss, model.trainable_weights) #inform loss and weights
    if False: #enable with True in case you want to see the objects
        print("List of tensors representing the symbolic gradients:")
        for i in range(len(gradients)):
            print('symbolic gradient[',i,']=',gradients[i])
    print('Network input [x1, x2]:\n', inputs)
    print('Network output [out_o1, out_o2]:\n', outputs)
    print("targets:\n", targets)
    #show weights at the beginning of iteration s
    print('weights at the beginning of this step')
    for i in range(len(model.trainable_weights)):
        #note that model.trainable_weights[i] is an object of
        # <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable
        #therefore (see e.g. https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable )
        #you need to obtain its value via a TF session:
        print('weights[',i,']=',sess.run(model.trainable_weights[i]))
    print("MSE with the initial weights:", mse)

    print("2) ===After applying GRADIENT:===")
    print("------------- Results for step (iteration) =", s)
    # ===== Calculate numerical gradient from symbolic gradients =====
    # evaluated_gradients is a list, show its contents:
    evaluated_gradients = sess.run(gradients, feed_dict={model.input: inputs})
    print('Gradients g to be used in new_weights = current_weights - learning_rate*g')
    for i in range(len(evaluated_gradients)):
        print('gradients[',i,']=',evaluated_gradients[i])

    # Apply ("step down") the gradient for each layer, subtracting the gradients
    # from current weights scaled by the learning rate:
    for i in range(len(model.trainable_weights)):
        sess.run(tf.compat.v1.assign_sub(model.trainable_weights[i], learning_rate*evaluated_gradients[i]))

    #show weights after gradient propagation of iteration s
    print('weights after gradient propagation in this step')
    for i in range(len(model.trainable_weights)):
        print('weights[',i,']=',sess.run(model.trainable_weights[i]))

    # print the MSE with new weights:
    outputs = model.predict(inputs)
    mse = mean_squared_error(targets, outputs)
    print("MSE with the new weights:", mse)

#Collect and show final results
final_outputs = model.predict(inputs)
final_mse = mean_squared_error(targets, final_outputs)

print("\n ===AFTER executing all GRADIENT descent steps===")
#show weights at the end of training
for i in range(len(model.trainable_weights)):
    #note that model.trainable_weights[i] is an object of
    # <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable
    #therefore (see e.g. https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable )
    #you need to obtain its value via a TF session:
    print('final weights[',i,']=',sess.run(model.trainable_weights[i]))
print("outputs:\n", final_outputs)
print("targets:\n", targets)
print("Final MSE = ", final_mse)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 2)                 6         
                                                                 
 dense_5 (Dense)             (None, 2)                 6         
                                                                 
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________
===START TRAINING THE NETWORK:===
Learning rate =  0.5
 ############ Step = 0 ############
1) ===BEFORE using any GRADIENT:===
Network input [x1, x2]:
 [[0. 1.]]
Network output [out_o1, out_o2]:
 [[0.65284914 0.6843307 ]]
targets:
 [[-1.  -1.3]]
weights at the beginning of this step
weights[ 0 ]= [[ 0.1  0.3]
 [-0.4  0.2]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


weights[ 1 ]= [-0.4  0.8]
weights[ 2 ]= [[ 0.4  0.2]
 [-0.4  0.7]]
weights[ 3 ]= [0.8 0.2]
MSE with the initial weights: 3.334739303243694
2) ===After applying GRADIENT:===
------------- Results for step (iteration) = 0
Gradients g to be used in new_weights = current_weights - learning_rate*g
gradients[ 0 ]= [[0.         0.        ]
 [0.05039085 0.0295356 ]]
gradients[ 1 ]= [0.05039085 0.0295356 ]
gradients[ 2 ]= [[0.11613464 0.13289538]
 [0.27385238 0.3133752 ]]
gradients[ 3 ]= [0.374597   0.42865947]
weights after gradient propagation in this step
weights[ 0 ]= [[ 0.1         0.3       ]
 [-0.42519543  0.1852322 ]]
weights[ 1 ]= [-0.42519543  0.7852322 ]
weights[ 2 ]= [[ 0.34193268  0.13355231]
 [-0.5369262   0.5433124 ]]
weights[ 3 ]= [ 0.61270154 -0.01432973]
MSE with the new weights: 3.0607885072991907
Learning rate =  0.5
 ############ Step = 1 ############
1) ===BEFORE using any GRADIENT:===
Network input [x1, x2]:
 [[0. 1.]]
Network output [out_o1, out_o2]:
 [[0.5807064  0.6034

# Amostra 2 com rate 0.5

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from keras import backend as k
from keras import losses
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error

#Initialize all weights to match the values adopted in the example at
#https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/ 
#Initialise kernel (weights matrix) to required value
def kernel_init_layer1(shape,dtype='float'):
    # In this simple example, assume that the shape is (2, 2) 
    # and create a fixed array with this dimension for layer 1
    kernel = np.array([[0.1, 0.3],[-0.4, 0.2]])
    return kernel 

#Initialise kernel (weights matrix) to required value
def kernel_init_layer2(shape,dtype='float'):
    # In this simple example, assume that the shape is (2, 2)
    # and create a fixed array with this dimension for layer 2
    kernel = np.array([[0.4, 0.2], [-0.4, 0.7]])
    return kernel 

#Initialise kernel (bias vector) to required value
def kernel_init_bias1(shape,dtype='float'):
    # In this simple example, assume that the shape is (1, 2)
    # and create a fixed array with this dimension for the 
    # bias vector of layer 1
    kernel = np.array([-0.4, 0.8]) #both neurons with the same value
    return kernel 

#Initialise kernel (bias vector) to required value
def kernel_init_bias2(shape,dtype='float'):
    # In this simple example, assume that the shape is (1, 2)
    # and create a fixed array with this dimension for the 
    # bias vector of layer 2
    kernel = np.array([0.8, 0.2]) #both neurons with the same value
    return kernel 

# from https://stackoverflow.com/questions/66221788/tf-gradients-is-not-supported-when-eager-execution-is-enabled-use-tf-gradientta
# TF 2 does not use "eager" execution, so disable it:
tf.compat.v1.disable_eager_execution()

#Define the neural network model with dense layers. Syntax:
#https://keras.io/api/layers/core_layers/dense/
#The sigmoid activation function in Keras is the standard logistic function 1/(1+exp(-x)).
#https://keras.io/api/layers/activations/
model = Sequential()
model.add(Dense(2, input_dim=2, use_bias=True,  bias_initializer=kernel_init_bias1,
        kernel_initializer=kernel_init_layer1, activation='sigmoid'))
model.add(Dense(2, use_bias=True,  bias_initializer=kernel_init_bias2, 
        kernel_initializer=kernel_init_layer2, activation='sigmoid'))
model.summary() # display the architecture

# We are not going to use a Keras optimizer. Define a learning rate:
learning_rate = 0.5  #you can change to 0.5 or any other reasonable value

#now compile the model, informing loss and performanc metrics that
#should be computed along the training
model.compile(loss='mse', metrics=['accuracy'])

# Begin TensorFlow
sess = tf.compat.v1.InteractiveSession()
sess.run(tf.compat.v1.initialize_all_variables())

model.optimizer.lr = learning_rate

print("===START TRAINING THE NETWORK:===")
steps = 3 # steps of gradient descent
for s in range(steps):
    print('Learning rate = ', k.get_value(model.optimizer.lr))
    print(" ############ Step = " + str(s) + " ############")
    print("1) ===BEFORE using any GRADIENT:===")
    #define input and target vectors, and also inform it to the loss function object
    #in this code we will keep the same inputs and targets along the steps, but the inputs and
    #targets can change when we go, for instance, through the training set
    inputs = np.array([[0.0,-1]]) #define the input for the current iteration (step)
    outputs = model.predict(inputs) #forward pass
    #define the target vector for the current iteration (step)
    targets = np.array([[1, 0.9]]) #notice that a sigmoid can output within range [0, 1]
    mse = mean_squared_error(targets, outputs) #calculate MSE
    #initialize loss object to be later incorporated to the gradients object that
    #enables the calculation of the symbolic gradients
    loss = losses.mean_squared_error(targets, model.output)
    #  ===== Obtain symbolic gradient to calculate numerical gradients =====
    gradients = k.gradients(loss, model.trainable_weights) #inform loss and weights
    if False: #enable with True in case you want to see the objects
        print("List of tensors representing the symbolic gradients:")
        for i in range(len(gradients)):
            print('symbolic gradient[',i,']=',gradients[i])
    print('Network input [x1, x2]:\n', inputs)
    print('Network output [out_o1, out_o2]:\n', outputs)
    print("targets:\n", targets)
    #show weights at the beginning of iteration s
    print('weights at the beginning of this step')
    for i in range(len(model.trainable_weights)):
        #note that model.trainable_weights[i] is an object of
        # <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable
        #therefore (see e.g. https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable )
        #you need to obtain its value via a TF session:
        print('weights[',i,']=',sess.run(model.trainable_weights[i]))
    print("MSE with the initial weights:", mse)

    print("2) ===After applying GRADIENT:===")
    print("------------- Results for step (iteration) =", s)
    # ===== Calculate numerical gradient from symbolic gradients =====
    # evaluated_gradients is a list, show its contents:
    evaluated_gradients = sess.run(gradients, feed_dict={model.input: inputs})
    print('Gradients g to be used in new_weights = current_weights - learning_rate*g')
    for i in range(len(evaluated_gradients)):
        print('gradients[',i,']=',evaluated_gradients[i])

    # Apply ("step down") the gradient for each layer, subtracting the gradients
    # from current weights scaled by the learning rate:
    for i in range(len(model.trainable_weights)):
        sess.run(tf.compat.v1.assign_sub(model.trainable_weights[i], learning_rate*evaluated_gradients[i]))

    #show weights after gradient propagation of iteration s
    print('weights after gradient propagation in this step')
    for i in range(len(model.trainable_weights)):
        print('weights[',i,']=',sess.run(model.trainable_weights[i]))

    # print the MSE with new weights:
    outputs = model.predict(inputs)
    mse = mean_squared_error(targets, outputs)
    print("MSE with the new weights:", mse)

#Collect and show final results
final_outputs = model.predict(inputs)
final_mse = mean_squared_error(targets, final_outputs)

print("\n ===AFTER executing all GRADIENT descent steps===")
#show weights at the end of training
for i in range(len(model.trainable_weights)):
    #note that model.trainable_weights[i] is an object of
    # <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable
    #therefore (see e.g. https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable )
    #you need to obtain its value via a TF session:
    print('final weights[',i,']=',sess.run(model.trainable_weights[i]))
print("outputs:\n", final_outputs)
print("targets:\n", targets)
print("Final MSE = ", final_mse)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 2)                 6         
                                                                 
 dense_7 (Dense)             (None, 2)                 6         
                                                                 
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________
===START TRAINING THE NETWORK:===


/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Learning rate =  0.5
 ############ Step = 0 ############
1) ===BEFORE using any GRADIENT:===
Network input [x1, x2]:
 [[ 0. -1.]]
Network output [out_o1, out_o2]:
 [[0.6773757 0.6796055]]
targets:
 [[1.  0.9]]
weights at the beginning of this step
weights[ 0 ]= [[ 0.1  0.3]
 [-0.4  0.2]]
weights[ 1 ]= [-0.4  0.8]
weights[ 2 ]= [[ 0.4  0.2]
 [-0.4  0.7]]
weights[ 3 ]= [0.8 0.2]
MSE with the initial weights: 0.07633009912304971
2) ===After applying GRADIENT:===
------------- Results for step (iteration) = 0
Gradients g to be used in new_weights = current_weights - learning_rate*g
gradients[ 0 ]= [[-0.         -0.        ]
 [ 0.00945002  0.00123317]]
gradients[ 1 ]= [-0.00945002 -0.00123317]
gradients[ 2 ]= [[-0.03525282 -0.02399455]
 [-0.04552241 -0.03098447]]
gradients[ 3 ]= [-0.07050563 -0.04798911]
weights after gradient propagation in this step
weights[ 0 ]= [[ 0.1         0.3       ]
 [-0.40472502  0.19938342]]
weights[ 1 ]= [-0.395275   0.8006166]
weights[ 2 ]= [[ 0.4176264   0.211

#Amostra 3 com rate 0.5

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from keras import backend as k
from keras import losses
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error

#Initialize all weights to match the values adopted in the example at
#https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/ 
#Initialise kernel (weights matrix) to required value
def kernel_init_layer1(shape,dtype='float'):
    # In this simple example, assume that the shape is (2, 2) 
    # and create a fixed array with this dimension for layer 1
    kernel = np.array([[0.1, 0.3],[-0.4, 0.2]])
    return kernel 

#Initialise kernel (weights matrix) to required value
def kernel_init_layer2(shape,dtype='float'):
    # In this simple example, assume that the shape is (2, 2)
    # and create a fixed array with this dimension for layer 2
    kernel = np.array([[0.4, 0.2], [-0.4, 0.7]])
    return kernel 

#Initialise kernel (bias vector) to required value
def kernel_init_bias1(shape,dtype='float'):
    # In this simple example, assume that the shape is (1, 2)
    # and create a fixed array with this dimension for the 
    # bias vector of layer 1
    kernel = np.array([-0.4, 0.8]) #both neurons with the same value
    return kernel 

#Initialise kernel (bias vector) to required value
def kernel_init_bias2(shape,dtype='float'):
    # In this simple example, assume that the shape is (1, 2)
    # and create a fixed array with this dimension for the 
    # bias vector of layer 2
    kernel = np.array([0.8, 0.2]) #both neurons with the same value
    return kernel 

# from https://stackoverflow.com/questions/66221788/tf-gradients-is-not-supported-when-eager-execution-is-enabled-use-tf-gradientta
# TF 2 does not use "eager" execution, so disable it:
tf.compat.v1.disable_eager_execution()

#Define the neural network model with dense layers. Syntax:
#https://keras.io/api/layers/core_layers/dense/
#The sigmoid activation function in Keras is the standard logistic function 1/(1+exp(-x)).
#https://keras.io/api/layers/activations/
model = Sequential()
model.add(Dense(2, input_dim=2, use_bias=True,  bias_initializer=kernel_init_bias1,
        kernel_initializer=kernel_init_layer1, activation='sigmoid'))
model.add(Dense(2, use_bias=True,  bias_initializer=kernel_init_bias2, 
        kernel_initializer=kernel_init_layer2, activation='sigmoid'))
model.summary() # display the architecture

# We are not going to use a Keras optimizer. Define a learning rate:
learning_rate = 0.5  #you can change to 0.5 or any other reasonable value

#now compile the model, informing loss and performanc metrics that
#should be computed along the training
model.compile(loss='mse', metrics=['accuracy'])

# Begin TensorFlow
sess = tf.compat.v1.InteractiveSession()
sess.run(tf.compat.v1.initialize_all_variables())

model.optimizer.lr = learning_rate

print("===START TRAINING THE NETWORK:===")
steps = 3 # steps of gradient descent
for s in range(steps):
    print('Learning rate = ', k.get_value(model.optimizer.lr))
    print(" ############ Step = " + str(s) + " ############")
    print("1) ===BEFORE using any GRADIENT:===")
    #define input and target vectors, and also inform it to the loss function object
    #in this code we will keep the same inputs and targets along the steps, but the inputs and
    #targets can change when we go, for instance, through the training set
    inputs = np.array([[1.2,0]]) #define the input for the current iteration (step)
    outputs = model.predict(inputs) #forward pass
    #define the target vector for the current iteration (step)
    targets = np.array([[1, 0.9]]) #notice that a sigmoid can output within range [0, 1]
    mse = mean_squared_error(targets, outputs) #calculate MSE
    #initialize loss object to be later incorporated to the gradients object that
    #enables the calculation of the symbolic gradients
    loss = losses.mean_squared_error(targets, model.output)
    #  ===== Obtain symbolic gradient to calculate numerical gradients =====
    gradients = k.gradients(loss, model.trainable_weights) #inform loss and weights
    if False: #enable with True in case you want to see the objects
        print("List of tensors representing the symbolic gradients:")
        for i in range(len(gradients)):
            print('symbolic gradient[',i,']=',gradients[i])
    print('Network input [x1, x2]:\n', inputs)
    print('Network output [out_o1, out_o2]:\n', outputs)
    print("targets:\n", targets)
    #show weights at the beginning of iteration s
    print('weights at the beginning of this step')
    for i in range(len(model.trainable_weights)):
        #note that model.trainable_weights[i] is an object of
        # <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable
        #therefore (see e.g. https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable )
        #you need to obtain its value via a TF session:
        print('weights[',i,']=',sess.run(model.trainable_weights[i]))
    print("MSE with the initial weights:", mse)

    print("2) ===After applying GRADIENT:===")
    print("------------- Results for step (iteration) =", s)
    # ===== Calculate numerical gradient from symbolic gradients =====
    # evaluated_gradients is a list, show its contents:
    evaluated_gradients = sess.run(gradients, feed_dict={model.input: inputs})
    print('Gradients g to be used in new_weights = current_weights - learning_rate*g')
    for i in range(len(evaluated_gradients)):
        print('gradients[',i,']=',evaluated_gradients[i])

    # Apply ("step down") the gradient for each layer, subtracting the gradients
    # from current weights scaled by the learning rate:
    for i in range(len(model.trainable_weights)):
        sess.run(tf.compat.v1.assign_sub(model.trainable_weights[i], learning_rate*evaluated_gradients[i]))

    #show weights after gradient propagation of iteration s
    print('weights after gradient propagation in this step')
    for i in range(len(model.trainable_weights)):
        print('weights[',i,']=',sess.run(model.trainable_weights[i]))

    # print the MSE with new weights:
    outputs = model.predict(inputs)
    mse = mean_squared_error(targets, outputs)
    print("MSE with the new weights:", mse)

#Collect and show final results
final_outputs = model.predict(inputs)
final_mse = mean_squared_error(targets, final_outputs)

print("\n ===AFTER executing all GRADIENT descent steps===")
#show weights at the end of training
for i in range(len(model.trainable_weights)):
    #note that model.trainable_weights[i] is an object of
    # <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable
    #therefore (see e.g. https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable )
    #you need to obtain its value via a TF session:
    print('final weights[',i,']=',sess.run(model.trainable_weights[i]))
print("outputs:\n", final_outputs)
print("targets:\n", targets)
print("Final MSE = ", final_mse)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 2)                 6         
                                                                 
 dense_9 (Dense)             (None, 2)                 6         
                                                                 
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________
===START TRAINING THE NETWORK:===


/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Learning rate =  0.5
 ############ Step = 0 ############
1) ===BEFORE using any GRADIENT:===
Network input [x1, x2]:
 [[1.2 0. ]]
Network output [out_o1, out_o2]:
 [[0.6609764 0.6940291]]
targets:
 [[1.  0.9]]
weights at the beginning of this step


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


weights[ 0 ]= [[ 0.1  0.3]
 [-0.4  0.2]]
weights[ 1 ]= [-0.4  0.8]
weights[ 2 ]= [[ 0.4  0.2]
 [-0.4  0.7]]
weights[ 3 ]= [0.8 0.2]
MSE with the initial weights: 0.07868050462676479
2) ===After applying GRADIENT:===
------------- Results for step (iteration) = 0
Gradients g to be used in new_weights = current_weights - learning_rate*g
gradients[ 0 ]= [[-1.1513641e-02 -4.9862119e-05]
 [-0.0000000e+00 -0.0000000e+00]]
gradients[ 1 ]= [-9.5947003e-03 -4.1551764e-05]
gradients[ 2 ]= [[-0.03270185 -0.01882739]
 [-0.05783894 -0.03329954]]
gradients[ 3 ]= [-0.07597064 -0.04373848]
weights after gradient propagation in this step
weights[ 0 ]= [[ 0.10575682  0.30002496]
 [-0.4         0.2       ]]
weights[ 1 ]= [-0.39520267  0.8000208 ]
weights[ 2 ]= [[ 0.41635093  0.20941369]
 [-0.37108055  0.7166498 ]]
weights[ 3 ]= [0.83798534 0.22186925]
MSE with the new weights: 0.07200185376402743
Learning rate =  0.5
 ############ Step = 1 ############
1) ===BEFORE using any GRADIENT:===
Network input [x